# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

<div class="span5 alert alert-info">
### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet


#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
</div>
****

In [122]:
import pandas as pd
import numpy as np
from scipy import stats

In [123]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')

In [124]:
# number of callbacks for black-sounding names
sum(data[data.race=='b'].call)

157.0

In [125]:
data.head()

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,b,1,3,3,6,0,1,1,0,316,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
2,b,1,4,1,6,0,0,0,0,19,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
3,b,1,3,4,6,0,1,0,1,313,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
4,b,1,3,3,22,0,0,0,0,313,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Nonprofit


# 1. A chi-square pearson test is applied to the data to assess whether race is a significaint driver of call back rates.

Alternatively a two-tailed test could be applied to test whether mean call back rates are statistically different. This test is not used here, but if it were CLT would apply. This test is applied below.

In [126]:
sig_level = 0.05

cont_table = pd.DataFrame(data=None, index=['black_name','white_name'], columns=['called','not_called'])
cont_table

,called,not_called
black_name,NaN,NaN
white_name,NaN,NaN


In [127]:
cont_table.loc["black_name","called"] = data[(data.race=='b') & (data.call==1)].shape[0]
cont_table.loc["black_name","not_called"] = data[(data.race=='b') & (data.call==0)].shape[0]
cont_table.loc["white_name","called"] = data[(data.race=='w') & (data.call==1)].shape[0]
cont_table.loc["white_name","not_called"] = data[(data.race=='w') & (data.call==0)].shape[0]
cont_table

,called,not_called
black_name,157,2278
white_name,235,2200


In [128]:
chi_s, p_val, dof, expctd = stats.chi2_contingency(cont_table)

# 2. The null hypothesis is that the call back rates for black and white sounding names is the same. The chi-squared results in a P-value < 1%, indicating that we can reject the null hypothesis

In [129]:
chi_s, p_val, dof, expctd 

(16.44902858418937, 4.9975783899632552e-05, 1, array([[  196.,  2239.],
        [  196.,  2239.]]))

# 3. Two tailed t-test is applied to call rates to calculate margin of error, confidence internval and p-values 

In [130]:
stats.ttest_ind(data[(data.race=='b')].call, data[(data.race=='w')].call)

Ttest_indResult(statistic=-4.1147052908617514, pvalue=3.9408021031288859e-05)

In [131]:
b_cr_mean = np.mean(data[(data.race=='b')].call)
w_cr_mean = np.mean(data[(data.race=='w')].call)
b_cr_std = np.std(data[(data.race=='b')].call)
w_cr_std = np.std(data[(data.race=='w')].call)
n1 = len(data[(data.race=='b')].call)
n2 = len(data[(data.race=='w')].call)

mean_diff = b_cr_mean - w_cr_mean
stand_err = np.sqrt((b_cr_std**2)/n1 + (w_cr_std**2)/n2)
p_val = mean_diff / stand_err

In [132]:
# Sample size is large enough for z statistic to be valid. P-value is close to that based on t-statistics
p_val

-4.1155834220829677

In [133]:
std_error_dist = stats.norm(loc = mean_diff, scale = stand_err)

lower_bound, upper_bound = std_error_dist.ppf(sig_level/2), std_error_dist.ppf(1-sig_level/2)

print ('There is a {0:.0f}% probability that call back rates for people with black'
       ' with black sounding names is between {1:.1%} and {2:.1%} lower').format((1-sig_level)*100,
                                                                                                lower_bound,
                                                                                                upper_bound)

There is a 95% probability that call back rates for people with black with black sounding names is between -4.7% and -1.7% lower


# Results show that whether resumes have black or white sounding names, without considering other factors, is correlated to call back rates. However this analysis does not determine wether we are observing causation.

In order to ascertain whether it is names, as opposed to another variable correlated with names in the sample population, other potential factors would need to be elimiated